pip install pandas scikit-learn numpy transformers torch tqdm sentence-transformers


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, classification_report
import numpy as np
from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder



C:\Users\Jet\anaconda3\envs\cud1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Цель**: предсказание пола и возрастной категории пользователей на основе их истории просмотров видео. Используются:
- Заголовки видео
- Демографические данные
- Технические характеристики устройств

## Используемые библиотеки
- `sentence-transformers` — для генерации эмбеддингов заголовков.
- `CatBoostClassifier` — для обучения модели с категориальными признаками.

## Этапы работы
1. **Загрузка и объединение данных**  
   Объединение таблиц с данными о просмотрах, видео и целевых переменных.
   
2. **Очистка заголовков**  
   Функция `clean_title` обрабатывает заголовки видео, удаляя лишние символы и ненужные слова для повышения качества предсказаний.

3. **Генерация эмбеддингов заголовков**  
   Модель `SentenceTransformer` преобразует заголовки видео в числовые эмбеддинги, которые используются для обучения модели.

4. **Агрегация данных по пользователям**  
   Для каждого пользователя агрегируются средние эмбеддинги заголовков, а также другие характеристики (время просмотра, технические данные).

5. **Обучение модели**  
   Модель `CatBoostClassifier` обучается на данных с категориальными признаками для предсказания пола и возрастной категории.

6. **Оценка модели**  
   Метрики точности и F1 выводятся для оценки качества работы модели.

# Результаты предсказания

## Предсказание возрастной категории
**F1 Score (weighted)**: `0.4541`

## Предсказание пола
**F1 Score (weighted)**: `0.7480`


In [2]:
# Очистка заголовков
key_phrases = list(set(["Женский стендап", "Comedy", "Мстители", "Мужское Женское", "На ножах", "Однажды в России",
                        "Очень странные дела", "Пацаны", "Свадьба", "Свидание", "Сумерки", "Теория Большого взрыва",
                        "Экстрасенсы", "Во все тяжкие", "Мужское / Женское", "ТРУШНЫЙ", "Эмили в Париже", "Шерлок",
                        "Четыре жены", "Черный список", "Человек-паук", "Человек-невидимка", "Хороший доктор",
                        "Властелин", "Веном", "Ван-Пис", "Бумажный", "Бриджертоны", "БЕРЕМЕННА", "Аркейн", "Аквамен",
                        "Аватар", "Секрет на миллион", "Пожалуйста не рассказывай!", "Морские дьяволы", "За гранью",
                        "ДНК", "Следствие вели", "Comedy Club", "Stand Up", "Ходячие мертвецы", "ТНТ", "Трансформеры",
                        "Сёгун", "МАМА", "Сумеречные охотники", "Одни из нас", "Пацанки", "Острые козырьки",
                        "Маша и Медведь", "Молодые ножи", "Звёздные войны", "Пучков", "Gravity Falls", "Гравити Фолз",
                        "Гарри Поттер","Аниме","Fallout"]))

# Приводим все ключевые фразы к нижнему регистру для удобства сравнения
key_phrases_lower = [phrase.lower() for phrase in key_phrases]

In [32]:
all_events = pd.read_csv('all_events.csv')
train_events = pd.read_csv('train_events.csv')
video_info = pd.read_csv('video_info_v2.csv')
train_targets = pd.read_csv('train_targets.csv')

# Объединение данных
train_data = pd.merge(train_events, train_targets, on='viewer_uid', how='left')
train_data = pd.merge(train_data, video_info, on='rutube_video_id', how='left')
all_events = pd.merge(all_events, video_info, on='rutube_video_id', how='left')

# Разделение на обучающие и тестовые данные
unique_viewers = train_data['viewer_uid'].unique()
train_viewers, test_viewers = train_test_split(unique_viewers, test_size=0.2, random_state=42)
train_df = train_data[train_data['viewer_uid'].isin(train_viewers)].reset_index(drop=True)
test_df = train_data[train_data['viewer_uid'].isin(test_viewers)].reset_index(drop=True)

# Проверка на пересечения пользователей
intersection = set(train_viewers).intersection(set(test_viewers))
if len(intersection) == 0:
    print("Пересечений пользователей между train и test нет")
else:
    print(f"Предупреждение: {len(intersection)} пересекающихся пользователей между train и test")

# Категориальные признаки
categorical_features = ['region', 'ua_device_type', 'ua_client_type', 'ua_os',
                        'ua_client_name', 'rutube_video_id', 'author_id', 'category']

categorical_mapping = {}
for col in categorical_features:
    unique_values = train_df[col].astype(str).unique().tolist()
    mapping = {val: idx for idx, val in enumerate(unique_values)}
    categorical_mapping[col] = mapping
    train_df[col] = train_df[col].astype(str).map(mapping)
    test_df[col] = test_df[col].astype(str).map(mapping).fillna(len(mapping)).astype(int)

 

def clean_title(title):
    # Приводим заголовок к нижнему регистру для корректного сравнения
    title_lower = title.lower()

    # Список для хранения всех найденных фраз
    found_phrases = []

    # Проверяем, если заголовок содержит одну из ключевых фраз
    for phrase, phrase_lower in zip(key_phrases, key_phrases_lower):
        if phrase_lower in title_lower:
            found_phrases.append(phrase)  # Добавляем оригинальную ключевую фразу в список

    # Если найдены ключевые фразы, возвращаем их через запятую
    if found_phrases:
        return ', '.join(found_phrases)

    # Обработка для ASMR
    if 'асмр' in title_lower or 'asmr' in title_lower:
        return 'АСМР'
    
    # Общая очистка для других заголовков
    title = re.sub(r'(сезон|выпуск|серия|финал|день|часть|ФИНАЛ|Серия|Новогодний)', '', title, flags=re.IGNORECASE)
    title = title.replace('Новая Битва экстрасенсов', 'Экстрасенсы').replace('экстрасенсов', 'Экстрасенсы')
    title = title.replace('Битва', '').replace('сильнейших', '').replace('шефов .', 'шеф')
    title = title.replace('Битва', '').replace('сильнейших', '')  # Удаление "Битва сильнейших"
    title = title.replace('шефов .', 'шеф')
    # Удаление лишних пробелов и символов
    title = re.sub(r'\d+', '', title)  # Удаление цифр
    title = re.sub(r'[^\w\s]', '', title)  # Удаление спецсимволов
    title = re.sub(r'\s+', ' ', title)  # Удаление лишних пробелов

    # Возвращаем очищенный заголовок
    return title.strip()



# Генерация эмбеддингов
def generate_embeddings(clean_df, batch_size=8):
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    
    embeddings = []
    for i in tqdm(range(0, len(clean_df), batch_size), desc="Processing batches"):
        batch = clean_df['cleaned_title'][i:i + batch_size].tolist()
        batch_embeddings = model.encode(batch, device=device)
        embeddings.append(batch_embeddings)
    embeddings = np.vstack(embeddings)
    clean_df['title_embedding1'] = list(embeddings)
    return clean_df

# Агрегация данных по пользователям
def aggregate_user_data(clean_df):
    user_embeddings = clean_df.groupby('viewer_uid')['title_embedding1'].apply(lambda x: np.mean(x.tolist(), axis=0))
    user_info = clean_df.groupby('viewer_uid').agg({
        'age': 'max' if 'age' in clean_df.columns else None,
        'sex': 'first' if 'sex' in clean_df.columns else None,
        'region': lambda x: x.mode()[0],
        'ua_device_type': lambda x: x.mode()[0],
        'ua_client_type': lambda x: x.mode()[0],
        'ua_os': lambda x: x.mode()[0],
        'ua_client_name': lambda x: x.mode()[0],
        'total_watchtime': 'sum',
        'category': lambda x: x.mode()[0],
        'duration': 'sum',
        'age_class': (lambda x: x.mode()[0]) if 'age_class' in clean_df.columns else None,
    }).reset_index()
    user_info['mean_title_embedding'] = user_embeddings.values
    return user_info

# Обучение модели
def train_model(X_train, y_train):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model

# Оценка модели
def predict_and_evaluate(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1_weighted = f1_score(y_test, y_pred, average='weighted')
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score (weighted): {f1_weighted}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

In [5]:
train_df['cleaned_title'] = train_df['title'].apply(clean_title)
test_df['cleaned_title'] = test_df['title'].apply(clean_title)

# Кодирование пола
le_sex = LabelEncoder()
train_df['sex'] = le_sex.fit_transform(train_df['sex'])
test_df['sex'] = le_sex.transform(test_df['sex'])  # Применение обученного на train LabelEncoder

# Преобразование возраста
train_df['age_class'] = train_df['age_class'].astype(int)
test_df['age_class'] = test_df['age_class'].astype(int)

In [10]:
clean_df = generate_embeddings(train_df, batch_size=256)
test_df = generate_embeddings(test_df, batch_size=256)


clean_train_df = aggregate_user_data(clean_df)
clean_test_df = aggregate_user_data(test_df)

C:\Users\Jet\anaconda3\envs\cud1\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Processing batches: 100%|██████████████████████████████████████████████████████████| 5511/5511 [13:21<00:00,  6.88it/s]
C:\Users\Jet\anaconda3\envs\cud1\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Processing batches: 100%|█████████████████████████████████████████████████████

In [93]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np
import pandas as pd

# Подготовка данных
def prepare_data(df, target_column):
    X = df.drop(columns=['age_class', 'sex', 'age', 'viewer_uid', 'mean_title_embedding'])
    embeddings = np.vstack(df['mean_title_embedding'].values)
    X = pd.concat([X, pd.DataFrame(embeddings)], axis=1)
    y = df[target_column]
    return X, y

# Определение категориальных признаков (если они есть)
def get_categorical_features_indices(X):
    return np.where(X.dtypes == 'object')[0]

# Обучение модели с использованием CatBoost
def train_model_catboost(X_train, y_train, cat_features):
    
    model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, 
                               cat_features=cat_features, verbose=100, random_seed=42)
    model.fit(X_train, y_train)
    return model

# Оценка модели
def predict_and_evaluate_catboost(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1_weighted = f1_score(y_test, y_pred, average='weighted')
    
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score (weighted): {f1_weighted}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

# Универсальная функция для обучения и тестирования
def train_and_evaluate(df_train, df_test, target_column):
    # Подготовка данных
    X_train, y_train = prepare_data(df_train, target_column)
    X_test, y_test = prepare_data(df_test, target_column)
    
    # Определение категориальных признаков
    categorical_features_indices = get_categorical_features_indices(X_train)
    
    # Обучение модели
    model = train_model_catboost(X_train, y_train, categorical_features_indices)
    
    # Оценка модели
    predict_and_evaluate_catboost(model, X_test, y_test)

# Пример использования для предсказания возраста
model_age = train_and_evaluate(clean_train_df, clean_test_df, 'age_class')

# Пример использования для предсказания пола
model_sex = train_and_evaluate(clean_train_df, clean_test_df, 'sex')


0:	learn: 1.3526715	total: 232ms	remaining: 3m 52s
100:	learn: 1.1106214	total: 21.1s	remaining: 3m 7s
200:	learn: 1.0883944	total: 41.8s	remaining: 2m 46s
300:	learn: 1.0737246	total: 1m 1s	remaining: 2m 23s
400:	learn: 1.0618193	total: 1m 22s	remaining: 2m 2s
500:	learn: 1.0518750	total: 1m 41s	remaining: 1m 41s
600:	learn: 1.0424975	total: 2m 1s	remaining: 1m 20s
700:	learn: 1.0332533	total: 2m 22s	remaining: 1m
800:	learn: 1.0245570	total: 2m 43s	remaining: 40.6s
900:	learn: 1.0164232	total: 3m 3s	remaining: 20.2s
999:	learn: 1.0085103	total: 3m 22s	remaining: 0us
Accuracy: 0.46579451712357306
F1 Score (weighted): 0.4541013123560947
Classification Report:
              precision    recall  f1-score   support

           0       0.28      0.01      0.01      1578
           1       0.49      0.58      0.53     12552
           2       0.42      0.43      0.43     13070
           3       0.49      0.44      0.46      8803

    accuracy                           0.47     36003
   mac

In [84]:
def aggregate_user_data(clean_df):
    user_embeddings = clean_df.groupby('viewer_uid')['title_embedding1'].apply(lambda x: np.mean(x.tolist(), axis=0))
    user_info = clean_df.groupby('viewer_uid').agg({

        'region': lambda x: x.mode()[0],
        'ua_device_type': lambda x: x.mode()[0],
        'ua_client_type': lambda x: x.mode()[0],
        'ua_os': lambda x: x.mode()[0],
        'ua_client_name': lambda x: x.mode()[0],
        'total_watchtime': 'sum',
        'category': lambda x: x.mode()[0],
        'duration': 'sum',

    }).reset_index()
    user_info['mean_title_embedding'] = user_embeddings.values
    return user_info

def pred(train_data,model):
    
    categorical_features = [
        'region', 'ua_device_type', 'ua_client_type', 'ua_os',
        'ua_client_name', 'rutube_video_id', 'author_id', 'category'
    ]
    categorical_mapping = {}
    num_unique = {}
    for col in categorical_features:
        unique_values = train_data[col].astype(str).unique().tolist()
        mapping = {val: idx for idx, val in enumerate(unique_values)}
        categorical_mapping[col] = mapping
        num_unique[col] = len(mapping) + 1  
        train_data[col] = train_data[col].astype(str).map(mapping)

    
    
    
    
    train_data['cleaned_title'] = train_data['title'].apply(clean_title)
    le_sex = LabelEncoder()
    clean_df = generate_embeddings(train_data, batch_size=256)
    clean_train_df = aggregate_user_data(clean_df)
    X_train_embeddings = np.vstack(clean_train_df['mean_title_embedding'].values)
    X_train = pd.concat([clean_train_df.drop(columns=['mean_title_embedding']), pd.DataFrame(X_train_embeddings)], axis=1)
    y_pred = model.predict(X_train)

    return   y_pred,clean_train_df




In [85]:
all_events = pd.read_csv('all_events.csv')
train_events = pd.read_csv('train_events.csv')
video_info = pd.read_csv('video_info_v2.csv')
train_targets = pd.read_csv('train_targets.csv')

# Объединение данных
train_data = pd.merge(train_events, train_targets, on='viewer_uid', how='left')
train_data = pd.merge(train_data, video_info, on='rutube_video_id', how='left')

train_data =train_data.head(1000)

train_data= train_data.drop(columns=['age_class', 'sex', 'age'])


In [86]:
data_sex,client_df= pred(train_data,model_sex)
data_age,client_df= pred(train_data,model_age)

C:\Users\Jet\anaconda3\envs\cud1\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Processing batches: 100%|████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  7.36it/s]
C:\Users\Jet\anaconda3\envs\cud1\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Processing batches: 100%|█████████████████████████████████████████████████████

In [91]:
print("Размерность y_pred_sex:", data_sex.shape)
print("Размерность y_pred_age_class:", data_age.shape)
print("Размерность viewer_uid:", client_df['viewer_uid'].shape)

# Преобразование предсказаний в одномерные массивы (если они многомерные)
y_pred_sex = data_sex.flatten() if len(data_sex.shape) > 1 else data_sex
y_pred_age_class = data_age.flatten() if len(data_age.shape) > 1 else data_age
viewer_uid = client_df['viewer_uid'].values.flatten() if len(client_df['viewer_uid'].shape) > 1 else client_df['viewer_uid'].values

# Убедимся, что все массивы имеют одинаковую длину
assert len(viewer_uid) == len(y_pred_sex) == len(y_pred_age_class), "Размерности данных не совпадают!"

# Формирование датафрейма с результатами
result_df = pd.DataFrame({
    'viewer_uid': viewer_uid,  # Убедитесь, что это одномерный массив
    'sex': y_pred_sex,  # Предсказанные значения пола
    'age_class': y_pred_age_class  # Предсказанные значения возрастной категории
})

Размерность y_pred_sex: (738,)
Размерность y_pred_age_class: (738, 1)
Размерность viewer_uid: (738,)
